#### Bring in Required Libraries

In [1]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [34]:
# ODBC Connection for FileMaker
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor_fm = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 7.744 sec


In [15]:
# SQLAlchemy connection for SQL Server
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn_sql = engine.connect()

In [3]:
# Define SQL queries
sql_screws = "SELECT * FROM autodata.eng.tblRegScrew"
sql_inv = """
    SELECT
        dbo.tblInventory.EngPartNum as partNum,
        '[' + COALESCE(dbo.tblInventory.Cabinet, '') + ' ' + COALESCE(dbo.tblInventory.Drawer, '') + ' ' +
            CONVERT(VARCHAR, COALESCE(dbo.tblInventory.OnHand, 0)) + ']' AS Status
    FROM
        dbo.tblInventory
"""

try:
    # Ensure connection is valid before executing queries
    assert conn_sql is not None, "Database connection (conn_sql) is not defined or invalid."

    # Read data from SQL queries
    df_scr = pd.read_sql(sql_screws, conn_sql)
    df_inv = pd.read_sql(sql_inv, conn_sql)

    # Confirm required columns exist before proceeding (defensive programming)
    assert 'T_CONE PIN' in df_scr.columns, "Column 'T_CONE PIN' is missing in df_scr."
    assert 'partNum' in df_inv.columns, "Column 'partNum' is missing in df_inv."

    # Perform a lookup using pandas merge
    df_scr = pd.merge(
        df_scr, df_inv[['partNum', 'Status']],
        left_on='T_CONE PIN',
        right_on='partNum',
        how='left'
    )

    # Drop unnecessary key column (if needed)
    if 'partNum' in df_scr.columns:
        df_scr.drop(columns=['partNum'], inplace=True)

    # Convert DataFrame dtypes
    df_scr.fillna('None', inplace=True)
    df_db = df_scr.convert_dtypes()

    # Write merged data back to the SQL table
    df_db.to_sql('tblRegScrStatus', conn_sql, schema='eng', if_exists='replace', index=False)

    # Return or display the final DataFrame
    print(df_db.dtypes)

except Exception as e:
    print(f"Error: {e}")

finally:
    try:
        conn_sql.close()
    except Exception as e:
        print(f"Error while closing connection: {e}")

Screw Part Number                  string
Screw Description                  string
Material Part Number               string
Material Information::Grade        string
Clamp Type                         string
                                    ...  
T_ROLLING T. PLATE STATIORY_2nd    string
T_ROLLING DIE_2nd                  string
T_ROLLING T. PLATE MOVABLE_2nd     string
T_Dwg No                           string
Status                             string
Length: 61, dtype: object


In [3]:
# ODBC Connection for AS400
CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""
dbcnxn = pyodbc.connect(CONNAS400)
cursor_as400 = dbcnxn.cursor()

In [4]:
sql_spares = """
    SELECT PROD.FPSPRMAST1.SPH_PART,
        STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
        STRIP(PROD.FPSPRMAST1.SPH_DESC1),
        STRIP(PROD.FPSPRMAST1.SPH_DESC2),
        STRIP(PROD.FPSPRMAST1.SPH_MFG),
        STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
        STRIP(PROD.FPSPRMAST2.SPD_CABINT),
        STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
        PROD.FPSPRMAST2.SPD_QOHCUR,
        PROD.FPSPRMAST1.SPH_CURSTD,
        STRIP(PROD.FPSPRMAST2.SPD_REODTE),
        STRIP(PROD.FPSPRMAST2.SPD_USECC),
        STRIP(PROD.FPSPRMAST2.SPD_PURCC),
        STRIP(PROD.FPSPRMAST2.SPD_QREORD)
    FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
    WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))
"""

In [10]:
sql_parts = """
    SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
        "Screw Part Number" AS Screw, "Band Feed from Band data",
        "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
        "CameraInspectionRequired", "ScrDrvChk", "Cutout1", "DrawingPrintNumber","Size"
    FROM tbl8Tridon
"""

In [35]:
sql_bands = """
    SELECT "Band Part Number", "Feed Length","CutoutA Tool Number","CutoutB Tool Number","CutoutC Tool Number",
        "Dim A", "Dim B","Dim C", "Process", "Description", "Number of Notches", "Abutment Punch", "A Notches Removed",
        "B Notches Removed"
    FROM BANDS
"""

### Execute Spare Part Query on AS400

In [5]:
start = timer()
cursor_as400.execute(sql_spares)
result_spares = cursor_as400.fetchall()
print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

AS400 Connect/Query Time = 2.71 sec


### Execute Clamp Query on SQL Server

In [11]:
# Clamp Data
start = timer()
cursor_fm.execute(sql_parts)
result_clamp = cursor_fm.fetchall()
print("FileMaker Clamp Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Clamp Query Time = 43.279 sec


### Execute Band Query on FileMaker Server

In [36]:
# Band Data
start = timer()
cursor_fm.execute(sql_bands)
result_band = cursor_fm.fetchall()
print("FileMaker Band Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Band Query Time = 10.168 sec


#### Build Dataframe for Spare Parts

In [92]:
data_type_dict = {'StandardCost' : float, 'OnHand' : int, 'PartNum' : str, 'ReOrderPt' : int, 'ReOrderDate' : int}
df_spares = pd.DataFrame.from_records(result_spares)
df_spares.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand', 'StandardCost','ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
df_spares = df_spares.dropna()
df_spares = df_spares.astype(data_type_dict)
df_spares = df_spares.convert_dtypes()

df_obs = df_spares[df_spares.Cabinet.str.contains('OBS')]
df_obs_yest = pd.read_csv('c:\\temp\yesterday_obs.csv',header=None, sep='\t')
df_obs_yest.reset_index(drop=True, inplace=True)
df_obs.reset_index(drop=True, inplace=True)
df_obs_yest = df_obs_yest.fillna("")

df_obs_yest.columns = df_spares.columns
df_obs_yest = df_obs_yest.astype(data_type_dict)
df_obs_yest = df_obs_yest.convert_dtypes()

if not df_obs['PartNum'].equals(df_obs_yest['PartNum']):
    df_diff = pd.concat([df_obs, df_obs_yest]).drop_duplicates(keep=False)
    df_obs.to_csv('c:\\temp\yesterday_obs.csv', header=False, index=False, sep='\t')
    i = 0
    item_list = []
    for index, row in df_diff.iterrows():
        #print(row['PartNum'], row['EngPartNum'], row['Desc1'])
        item_list.append('<h5>Item ' + str(i) + '</h5>'
                             + '<p style="margin-left: 40px">'
                             + 'Part Number: ' + row['PartNum']
                             + '<br>Eng Part Number: <strong>' + row['EngPartNum'] + '</strong>'
                             + '<br>Description 1: ' + row['Desc1']
                             + '<br>Description 2: ' + row['Desc2']
                             + '<br>Manufacturer: ' + row['Mfg']
                             + '<br>Manufacturer Pn: <strong>' + row['MfgPn'] + '</strong>'
                             + '<br>Dept Use: ' + row['DeptUse']
                             + '<br>Dept Purch: ' + row['DeptPurch']
                             + '</p><br>')
        i += 1
    for item in item_list:
        print(item)  


<h5>Item 0</h5><p style="margin-left: 40px">Part Number: 1021446<br>Eng Part Number: <strong>091021446</strong><br>Description 1: PUNCH SUPPORT<br>Description 2: OBS ECR 14755<br>Manufacturer: <br>Manufacturer Pn: <strong></strong><br>Dept Use: 0750<br>Dept Purch: 0750</p><br>
<h5>Item 1</h5><p style="margin-left: 40px">Part Number: 1021447<br>Eng Part Number: <strong>091021447</strong><br>Description 1: OBS ECR 14755<br>Description 2: ****<br>Manufacturer: <br>Manufacturer Pn: <strong></strong><br>Dept Use: 0750<br>Dept Purch: 0750</p><br>
<h5>Item 2</h5><p style="margin-left: 40px">Part Number: 1021449<br>Eng Part Number: <strong>091021449</strong><br>Description 1: FINGER-<br>Description 2: ****<br>Manufacturer: <br>Manufacturer Pn: <strong></strong><br>Dept Use: 0750<br>Dept Purch: 0750</p><br>
<h5>Item 3</h5><p style="margin-left: 40px">Part Number: 1021450<br>Eng Part Number: <strong>091021450</strong><br>Description 1: FINGER<br>Description 2: <br>Manufacturer: <br>Manufacturer 

#### Build Dataframe for Clamp Data

In [28]:
df_clamps = pd.DataFrame.from_records(result_clamp)
# Set Column Names
df_clamps.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk', 'Cutout1', 'Drawing', 'Size']
# Set Data Types
data_type_dict = {'PartNumber' : str, 'Band' : str,'Housing' : str,'Screw' : str, 'Feed' : float, 'DiaMax' : float,'DiaMin' : float,'HexSz' : str,'BandThickness' : float, 'BandWidth' : float,'CamInspect' : str,'ScrDrvChk' : str,'Cutout1' : float,'Drawing' : str,'Size' : str}

# Do Some Filtering and Data Cleansing
df_clamps = df_clamps[df_clamps.Feed != 'N/A']
df_clamps = df_clamps[1:]
df_clamps = df_clamps.astype(data_type_dict)
df_clamps['CamInspect'] = df_clamps['CamInspect'].str.upper()
df_clamps['ScrDrvChk'] = df_clamps['ScrDrvChk'].str.upper()
df_clamps['Feed'] = df_clamps['Feed'].round(3)
df_clamps['BandWidth'] = df_clamps['BandWidth'].round(3)
df_clamps['BandThickness'] = df_clamps['BandThickness'].round(3)
df_clamps.fillna({'DiaMax' : 0.0}, inplace=True)
df_clamps.fillna({'DiaMin' : 0.0}, inplace=True)
df_clamps.fillna({'Cutout1' : 0.0}, inplace=True)
df_clamps['DiaMax'] = df_clamps['DiaMax'].round(3)
df_clamps['DiaMin'] = df_clamps['DiaMin'].round(3)
df_clamps['Cutout1'] = df_clamps['Cutout1'].round(3)
df_clamps['HexSz'] = df_clamps['HexSz'].replace('Purchased 5/16"', '5/16"')
df_clamps = df_clamps.dropna()
df_clamps = df_clamps[df_clamps.Size != 'None']
df_clamps = df_clamps.convert_dtypes()
df_clamps.sample(20)



,PartNumber,Band,Housing,Screw,Feed,DiaMax,DiaMin,HexSz,BandThickness,BandWidth,CamInspect,ScrDrvChk,Cutout1,Drawing,Size
10951,58560VB,7205600,7015400,2343003VB,13.312,0.0,0.0,None,0.022,0.562,NO,NO,0.0,58000,056
647,153032102,2314032,1063005,1213002,8.81,0.0,0.0,"5/16""",0.022,0.562,NO,NO,0.0,153-001,032
12633,D370140040001,2745040,2360013,D1278001MC,10.37,2.92,2.84,"5/16""",0.025,0.5,NO,NO,0.0,3701400XX001,040
5865,670040472052,2721472,2351045,1278001,95.072,0.0,0.0,"5/16""",0.025,0.5,NO,NO,0.0,67004_67-4,472
8283,670180236723,2721236,2367012,1278001,48.848,0.0,0.0,"5/16""",0.025,0.5,NO,NO,0.0,631-001,236
9604,670810061175,2247061,2360013,1377001MC,14.6,4.27,4.19,7 mm,0.022,0.5,YES,NO,3.55,67081 XXXX,061
11238,650040048052,79L4800,1019030,1278001,14.75,0.0,0.0,None,0.022,0.562,NO,NO,0.0,65004 XXXX,048
1419,M750014001,2827014,2367025,1073003,6.624,0.0,1.26,"5/16""",0.025,0.5,NO,NO,0.0,750-001,014
6323,379970042001,2662042,2360013,1300014MC,10.79,3.06,2.98,7 mm,0.025,0.5,YES,YES,0.0,3799700XX001,042
5119,370050036052,2745036,2360007,1299001MC,9.59,0.0,0.0,7 mm,0.025,0.5,NO,NO,0.0,37005 XXXX,036


#### Build Dataframe for Band Data

In [37]:
valid_process = ['IN-LINE SINGLE NOTCH DIE', 'IN-LINE DOUBLE NOTCH DIE', 'IN-LINE BAND STAMPING', 'IN-LINE 105 NOTCH DIE', 'CUT & CURL']
df_bands = pd.DataFrame.from_records(result_band)
df_bands.columns = ['Band', 'FeedLength', 'CutOutA', 'CutOutB', 'CutOutC', 'DimA', 'DimB', 'DimC', 'Process', 'Description','NumNotches', 'AbutmentPunch', 'ANotchesRemoved', 'BNotchesRemoved']
data_type_dict = {'Band' : str, 'FeedLength' : float,'CutOutA' : str,'CutOutB' : str,'CutOutC' : str,'DimA' : float,'DimB' : float,'DimC' : float,'Process' : str, 'Description' : str,'NumNotches' : float,'AbutmentPunch' : float,'ANotchesRemoved' : float,'BNotchesRemoved' : float}
df_bands = df_bands.astype(data_type_dict)
df_bands = df_bands.convert_dtypes()

# Shape Data Set
df_bands = df_bands[1:]
df_bands['Band'] = df_bands['Band'].str.upper()
df_bands['Process'] = df_bands['Process'].str.upper()
df_bands = df_bands[df_bands['Process'].isin(valid_process)]
df_bands = df_bands[df_bands['Band'].str.contains('OBS|X|OLD|PSR|CH|NO ACTIVE') == False]
df_bands['FeedLength'] = df_bands['FeedLength'].round(3)
df_bands.fillna({'CutOutA' : 'NON-MULTI'}, inplace=True)
df_bands.fillna({'DimA' : 0.000}, inplace=True)
df_bands.fillna({'DimB' : 0.000}, inplace=True)
df_bands.fillna({'DimC' : 0.000}, inplace=True)
df_bands.fillna({'NumNotches' : 0.000}, inplace=True)
df_bands.fillna({'AbutmentPunch' : 0.000}, inplace=True)
df_bands.fillna({'ANotchesRemoved' : 0.000}, inplace=True)
df_bands.fillna({'BNotchesRemoved' : 0.000}, inplace=True)
df_bands.drop_duplicates(subset='Band', keep='first', inplace=True)
df_bands = df_bands.dropna(subset='Band')

#df_bands.query('BandLength < 10.000', inplace=True)

df_bands.sample(20)
# df_bands.shape

,Band,FeedLength,CutOutA,CutOutB,CutOutC,DimA,DimB,DimC,Process,Description,NumNotches,AbutmentPunch,ANotchesRemoved,BNotchesRemoved
2146,2825758,14.45,091028398,None,None,5.179,0.0,0.0,IN-LINE SINGLE NOTCH DIE,"Band 1/2"" Side Notch",20,0,0,0
2938,2025016,5.66,None,None,None,5.252,0.0,0.0,CUT & CURL,"Band 1/2"" Setup",45,0,0,0
1460,2017037,9.663,None,None,None,0.329,0.313,5.538,CUT & CURL,Band Micro Setup,35,0,0,0
2352,2861747,11.93,091029117,091028259,091029117,2.402,1.488,0.373,IN-LINE SINGLE NOTCH DIE,"Band 1/2"" Dual Slot W/ RH Side Notch",17,0,0,0
1863,2017019,6.0,None,None,None,4.965,5.229,1.875,CUT & CURL,Band Micro Setup,35,0,0,0
9409,2966562,21.14,None,None,None,0.702,10.458,0.0,CUT & CURL,"5/8"" Band, Lined, Marked with the W4 markings",34,0,5,0
11059,2306852,12.68,None,None,None,0.0,0.0,0.0,IN-LINE DOUBLE NOTCH DIE,"9/16"", 301, .022",68,0,0,0
8910,2784046,13.5,091029420,091028259,091028259,4.878,1.456,8.61,CUT & CURL,ETDLHN=Ext Tang w/ Triangle & Dual Left Hand N...,23,0,0,0
3980,1053010,4.712,None,None,None,3.356,0.0,0.0,CUT & CURL,"Band 1/2"" Setup",15,0,0,0
3256,2353024,7.23,None,None,None,1.162,0.0,0.0,CUT & CURL,"Band 1/2"" Setup",37,0,0,0


#### Load Spare Parts to SQL Server Table

In [165]:
data_type_dict = {'StandardCost' : sqlalchemy.types.FLOAT, 'OnHand' : sqlalchemy.types.INT,'PartNum' : sqlalchemy.types.VARCHAR(255),'ReOrderPt' : sqlalchemy.types.INT, 'EngPartNum' : sqlalchemy.types.VARCHAR(255), 'Desc1' : sqlalchemy.types.VARCHAR(255),'Desc2' : sqlalchemy.types.VARCHAR(255),'Mfg' : sqlalchemy.types.VARCHAR(255),'MfgPn' : sqlalchemy.types.VARCHAR(255), 'Cabinet' : sqlalchemy.types.VARCHAR(255),'Drawer' : sqlalchemy.types.VARCHAR(255),'ReOrderDate' : sqlalchemy.types.VARCHAR(255),'DeptUse' : sqlalchemy.types.VARCHAR(255),'DeptPurch' : sqlalchemy.types.VARCHAR(255)}

df_spares.to_sql('tblSpares', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

28

#### Load Clamp to SQL Server Table

In [29]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR(255), 'Band' : sqlalchemy.types.VARCHAR(255),'Housing' : sqlalchemy.types.VARCHAR(255),'Screw' : sqlalchemy.types.VARCHAR(255), 'Feed' : sqlalchemy.types.Float, 'DiaMax' : sqlalchemy.types.Float,'DiaMin' : sqlalchemy.types.Float,'HexSz' : sqlalchemy.types.VARCHAR(255),'BandThickness' : sqlalchemy.types.Float, 'BandWidth' : sqlalchemy.types.Float,'CamInspect' : sqlalchemy.types.VARCHAR(255),'ScrDrvChk' : sqlalchemy.types.VARCHAR(255), 'Cutout1' : sqlalchemy.types.Float,'Drawing' : sqlalchemy.types.VARCHAR(255),'Size' : sqlalchemy.types.VARCHAR(255)}

df_clamps.to_sql('parts_clamps', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

18

#### Load Band Data to SQL Server Table

In [38]:
data_type_dict = {'Band' : sqlalchemy.types.VARCHAR(255),'FeedLength' : sqlalchemy.types.FLOAT, 'CutOutA' : sqlalchemy.types.VARCHAR(255), 'CutOutB' : sqlalchemy.types.VARCHAR(255), 'CutOutC' : sqlalchemy.types.VARCHAR(255),'DimA' : sqlalchemy.types.FLOAT,'DimB' : sqlalchemy.types.FLOAT,'DimC' : sqlalchemy.types.FLOAT, 'Process' : sqlalchemy.types.VARCHAR(255), 'Description' : sqlalchemy.types.VARCHAR(255),'NumNotches' : sqlalchemy.types.FLOAT,'AbutmentPunch' : sqlalchemy.types.FLOAT,'ANotchesRemoved' : sqlalchemy.types.FLOAT,'BNotchesRemoved' : sqlalchemy.types.FLOAT}

df_bands.to_sql('parts_bands', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

119